# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

all_files = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    for f in file_path_list:
        all_files.append(os.path.abspath(f))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in all_files:
    #print(f)
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6976


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Table 1:  Create the table that will provide results for a qeury returning the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4. 

## The query is searched by the given sessionId and itemInSession which also provide unique constraint for each row, hence the sessionId and itemInSession form the Primary Key of the table. 
## The artist, song title and length constitute the rest of the table columns as required by the query.
try:
    query1 = "CREATE TABLE IF NOT EXISTS songplays_session "
    query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length decimal,  PRIMARY KEY (sessionId, itemInSession))"
    session.execute(query1)
except Exception as e:
    print(e)

                    

In [11]:
## Insert data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO songplays_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## extract data from corresponding columns in each row
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## The querry that provides the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "select artist, song, length from songplays_session where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## The query is searched by userId and sessionId, hence userId and sessionId form the partition Key of the Primary to ensure the uniqueness of each row. 
## The itemInSession serves as clustering key to enforce the order"
## The artist, song title, firstName and lastName constitute the rest of the table columns as required by the query.
try:
    query2 = "CREATE TABLE IF NOT EXISTS user_songplays_session "
    query2 = query2 + "(userId int, sessionId int, iteminSession int, artist text, song text, firstName text, lastName text,PRIMARY KEY ((userId, sessionId), itemInSession)) "
    
    session.execute(query2)
except Exception as e:
    print(e)
                      

In [14]:
## Insert data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songplays_session (userId, sessionId,itemInSession, artist, song,   firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## extract data from corresponding columns in each row
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))

In [15]:
## Query providing name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select artist, song, itemInSession, firstName, lastName from user_songplays_session where userId = 10 and sessionId=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.iteminsession, row.firstname,row.lastname)


Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


In [16]:
## Create table for Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## The query is searched by song title and userId which also pribide sufficient constraint for unique identification for each row, hence userId and sessionId form the Primary Key. 
## The firstName and lastName constitute the rest of the table columns as required by the query.
try:
    query3 = "CREATE TABLE IF NOT EXISTS user_songplay "
    query3 = query3 + "(song text, user_id int, firstName text, lastName text, PRIMARY KEY (song, user_id)) "
    session.execute(query3)
except Exception as e:
    print(e)

                    

In [17]:
## Insert data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songplay (song, user_id,  firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## extract data from corresponding columns in each row
        session.execute(query, ( line[9], int(line[10]), line[1], line[4]))

In [18]:
## Query returns every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select firstName, lastName from user_songplay where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
## Drop the table before closing out the sessions

In [20]:
session.execute("drop table if exists music_app_history1")

In [21]:
session.execute("drop table if exists music_app_history2")

In [22]:
session.execute("drop table if exists music_app_history3")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()